In [2]:
import json
import pandas as pd

In [3]:
with open('reports/performance.json') as f:
    perf = json.load(f)

In [ ]:
for setting, perf_logs in perf.items():
    print(setting, "average perf under 10 runs")
    print(pd.DataFrame(perf_logs).mean())

gpt-4o-mini+no+context+no+reasoning average perf under 10 runs
accuracy     0.708537
precision    0.750435
recall       0.870175
f1           0.805576
dtype: float64
gpt-4o-mini+no+context average perf under 10 runs
accuracy     0.697642
precision    0.697526
recall       0.996427
f1           0.820590
dtype: float64
gpt-4o-mini+no+reasoning average perf under 10 runs
accuracy     0.762195
precision    0.755109
recall       0.973684
f1           0.850570
dtype: float64
gpt-4o-mini+context+reasoning average perf under 10 runs
accuracy     0.735080
precision    0.729875
recall       0.982456
f1           0.837503
dtype: float64


: 